# Content Warning

This project involves the use of lots of inappropriate and downright offensive/upsetting language. Some of the data analysis methods below print these texts to the notebook so they are visible to whoever opens it. Those who view the notebook please do so knowing this in advance. 

Topics: seixsm, racism, misogyny, and lots more unfortunately. 

# Logistic Regression

In this Jupyter notebook we will implement the logistical regression analysis of the text and attempt to see how well the data can be classified as hate speech. We will use this score to determine if logistical regression would be a good way to classify hate speech or not (as a tool).

We are following [this](https://www.geeksforgeeks.org/text-classification-using-logistic-regression/) tutorial.

In [3]:
try:
    print("Importing packages")
    import pandas as pd
    from sklearn.feature_extraction.text import CountVectorizer
    from sklearn.model_selection import train_test_split
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import accuracy_score, confusion_matrix
    print("Done importing packages")
except Exception as e:
    print("Error", e)

Importing packages
Done importing packages


### First, read in the data and analyze it

In [ ]:
dataframe = pd.read_csv("data/Ethos_Dataset_Binary.csv", sep=';', header=0)
print(dataframe.size)
dataframe.tail()

The data is arranged in two columns: "comment" and "isHate". The "isHate" column is a numerical column that has a float number between 0.0 and 1.0. If the number is 1.0 then the comment is classified as hate. If the number is 0.0 the comment is classified as not hate. 